In [1]:
import numpy as np
from numpy import random
from simanneal import Annealer
import nest
from scripts import optimization


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:53:34

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
nest.rng_seed = 1
np.random.seed(1)
nest.set_verbosity('M_FATAL')

In [3]:
experimental_data = np.load('/hpc/mzhu843/modelling/nest/DATA/processed/experiment3_unfiltered_processed.npy')
initial_connectivity_matrix = np.load('/hpc/mzhu843/modelling/nest/DATA/matrices/initial_matrix.npy')
lamb = 0.01

In [4]:
class OptimizationProblem(Annealer):
    def __init__(self, state, data, lamb):
        self.experimental_data = data
        self.lamb = lamb
        self.state = state

    def energy(self):
        results = optimization.simulate(self.state)[0]
        cost_function = optimization.ssd_with_l1(results, experimental_data, lamb, self.state)
        return cost_function

    def move(self):
        for i in range(10000):
            x = np.random.randint(0, 276)
            y = np.random.randint(0, 276)
            if self.state[x][y] != 0:
                self.state[x][y] = min(max(self.state[x][y] + np.random.uniform(-0.3, 0.3), 0.01), 2)

In [5]:
problem = OptimizationProblem(initial_connectivity_matrix, experimental_data, lamb)
problem.steps = 2000
x, func = problem.anneal()
np.save('/hpc/mzhu843/modelling/nest/results/optimization/optimised_weights', x)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
